### caio martins 180074741
### yuri alves 180078640

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.master("spark://cm1:7077").appName("testekafka").getOrCreate()

lines = spark.readStream.format("kafka") \
        .option("kafka.bootstrap.servers", "cm4:9092") \
        .option("subscribe", "student-a180074741-saida") \
        .option('includeTimestamp', 'true') \
        .load()

In [ ]:
from pyspark.sql.functions import explode, split, col
words = lines.select(explode(split(col("value"), "\s+")) \
        .alias("word"),lines.timestamp)

wordCounts = words.groupBy("word").count()
schema = (words.groupBy().count()).selectExpr("cast (count as string) total")

query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

count = schema \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()


words.writeStream.format('console').option('truncate', 'false').start()